C, GCC, Makefile, GDB
============================

- `C` is a compiled language, its programm file must have `.c` extension
- There are many `C` compilers, most popular ones for linux are
1) GCC
2) Clang
- In our course we will use `GCC`


In [34]:
%%file hello_world.c

#include <stdio.h>

int main() {
    printf("Hello world!\n");
    return 0;
}

Overwriting hello_world.c


- `-o` option is for giving name to the generated executable. If not used, binary file has name `a.out`.

In [35]:
!gcc hello_world.c -o hello_world.exe
!./hello_world.exe

Hello world!


## Compilation

- `C` programm compilation consists of __4__ main parts
  
---
1) Pre-processing __:__ comments removal, file inclusion, macros expansion, conditional compilation

2) Compilation __:__ convertion into assembly code

3) Assembling __:__ convertion into a machine-understandable code
   
4) Linking __:__ generates an executable file 
---
- [Small article about compilation](https://www.scaler.com/topics/c/compilation-process-in-c/)

### Pre-processing example

- Comments will be removed as they are not of particular use for the machine.
  
- File inclusion will cause the entire content of `header` to be added into the source code, replacing the `#include<header>`.
  
- Compile-time known constants, values or expressions defined using `#define` directive are replaced.
  
- Replace conditional cmpilation directives `#ifdef`, `#endif` with real codes.
-------
- `-E` option for generating file right after __pre-process__ step.

In [36]:
%%file preprocess_example.h
#define ERROR_CODE -1
#define SUCCESS_CODE 0

int convert_euros_to_rubles(int* amount) {
    if (*amount > 0) {
        *amount *= 60;
        return SUCCESS_CODE;
    }
    return ERROR_CODE;
}

Overwriting preprocess_example.h


In [37]:
%%file preprocess_example.c
#include "preprocess_example.h"

#define OPTION_1 1

// Some magic counting
int count() {
    int amount = 0;
    
    #ifdef OPTION_1
        amount = 33;
    #else
        amount = 77;
    #endif

    if (convert_euros_to_rubles(&amount) == 0) {
        return amount;
    }
    return ERROR_CODE;
}

Overwriting preprocess_example.c


In [38]:
!gcc -E preprocess_example.c -o preprocess_example_E.c

#### Code after prep-rocess step

```c
# 0 "preprocess_example.c"
# 0 "<built-in>"
# 0 "<command-line>"
# 1 "/usr/include/stdc-predef.h" 1 3 4
# 0 "<command-line>" 2
# 1 "preprocess_example.c"
# 1 "preprocess_example.h" 1



int convert_euros_to_rubles(int* amount) {
    if (*amount > 0) {
        *amount *= 60;
        return 0;
    }
    return -1;
}
# 2 "preprocess_example.c" 2




int count() {
    int amount = 0;


        amount = 33;




    if (convert_euros_to_rubles(&amount) == 0) {
        return amount;
    }
    return -1;
}
```

### Compilation


- Takes the output of the pre-processor and generates assembly language.
  
- It is an intermediate human readable language, __specific to the target processor__ .
  
- Assembly file must have `.S` extension.
----
- `-S` option for generating file right after __compilation__ step.

In [39]:
%%file asm_example.c
#include <limits.h>
#include <stdio.h>

unsigned int get_remains(unsigned int x, unsigned int y) {
    return x % y;
}

unsigned int sum(unsigned int x, unsigned int y) {
    if (x + y < x) {
        return UINT_MAX;
    }
    return x + y;
}

int main() {
    unsigned int x = 13;
    unsigned int y = 20;

    printf("The remainder of %u devided by %u is %u \n", x, y, get_remains(x, y));

    printf("Sum of %u and %u is %u \n", x, y, sum(x, y));

    printf("Sum of %u and 1 is %u which is the UINT_MAX \n", UINT_MAX, sum(1, UINT_MAX));

    return 0;
}

Overwriting asm_example.c


- Let's compile and look at first 20 and last 10 lines of assembly code

In [40]:
!gcc -S asm_example.c -o asm_example_S.S
!head -n 20 asm_example_S.S

	.file	"asm_example.c"
	.text
	.globl	get_remains
	.type	get_remains, @function
get_remains:
.LFB0:
	.cfi_startproc
	endbr64
	pushq	%rbp
	.cfi_def_cfa_offset 16
	.cfi_offset 6, -16
	movq	%rsp, %rbp
	.cfi_def_cfa_register 6
	movl	%edi, -4(%rbp)
	movl	%esi, -8(%rbp)
	movl	-4(%rbp), %eax
	movl	$0, %edx
	divl	-8(%rbp)
	movl	%edx, %eax
	popq	%rbp


In [41]:
!tail -n 10 asm_example_S.S

	.string	"GNU"
1:
	.align 8
	.long	0xc0000002
	.long	3f - 2f
2:
	.long	0x3
3:
	.align 8
4:


- Let's compile the file generated right after pre-process step.

In [42]:
!gcc -S -O1 preprocess_example_E.c -o preprocess_example_S.S

- `-O1` option stands for getting more compact and readable assembly file.
- We get valid assembly programm.

```asm
	.file	"preprocess_example_E.c"
	.text
	.globl	convert_euros_to_rubles
	.type	convert_euros_to_rubles, @function
convert_euros_to_rubles:
.LFB0:
	.cfi_startproc
	endbr64
	movl	(%rdi), %eax
	testl	%eax, %eax
	jle	.L3
	imull	$60, %eax, %eax
	movl	%eax, (%rdi)
	movl	$0, %eax
	ret
.L3:
	movl	$-1, %eax
	ret
	.cfi_endproc
.LFE0:
	.size	convert_euros_to_rubles, .-convert_euros_to_rubles
	.globl	count
	.type	count, @function
count:
.LFB1:
	.cfi_startproc
	endbr64
	movl	$1980, %eax
	ret
	.cfi_endproc
.LFE1:
	.size	count, .-count
	.ident	"GCC: (Ubuntu 11.2.0-19ubuntu1) 11.2.0"
	.section	.note.GNU-stack,"",@progbits
	.section	.note.gnu.property,"a"
	.align 8
	.long	1f - 0f
	.long	4f - 1f
	.long	5
0:
	.string	"GNU"
1:
	.align 8
	.long	0xc0000002
	.long	3f - 2f
2:
	.long	0x3
3:
	.align 8
4:
```

### Assembling

- Just generates from assembly code machine code file.

- It is not readable for humans.

- `-c` option generates machine code file and stops, __Linking__ step doesn't happen.

In [43]:
!gcc -c hello_world.c -o hello_world.obj
!cat hello_world.obj

                              9      0               k       '                             B                      �                                      R                     �                                     j                     �       8                             e      @               �                	                                      �       �                          	                      �                                                          �      t                                                        &                     ^                                      ,                    ^                                      1                     ^       

- `hello_world.obj` file is a machine code file, it is not readable in raw format as other files

- You can use `readelf -WaS` terminal command to get better picture.

In [44]:
!readelf -WaS hello_world.obj

ELF Header:
  Magic:   7f 45 4c 46 02 01 01 00 00 00 00 00 00 00 00 00 
  Class:                             ELF64
  Data:                              2's complement, little endian
  Version:                           1 (current)
  OS/ABI:                            UNIX - System V
  ABI Version:                       0
  Type:                              REL (Relocatable file)
  Machine:                           Advanced Micro Devices X86-64
  Version:                           0x1
  Entry point address:               0x0
  Start of program headers:          0 (bytes into file)
  Start of section headers:          608 (bytes into file)
  Flags:                             0x0
  Size of this header:               64 (bytes)
  Size of program headers:           0 (bytes)
  Number of program headers:         0
  Size of section headers:           64 (bytes)
  Number of section headers:         14
  Section header string table index: 13

Section Headers:
  [Nr] Name              Type  

### Linking

- The linking process generates an executable file.

- Includes library files to the programm.

- After linking the operating system uderstands where to look for library funtions, variables, etc.

- Linking process automatically takes place after __assembling__.
  
- `gcc` can build executable from simultaneously `.c` file `.S` and '.obj` files.

- You can link your source code file with `.obj` file from some library, and the linker will find the functions you use in your code in the library `.obj` file.

----------

#### Example

In [1]:
%%file gcd.h

int gcd(int x, int y);

Writing gcd.h


In [2]:
%%file gcd.c
#include "gcd.h"

int gcd(int x, int y) {
    while (x != y) {
        if (x > y) {
            x -= y;
        } else {
            y -= x;
        }
    }
    
    return x;
}

Writing gcd.c


In [5]:
%%file calculate_gcd.c
#include "gcd.h"
#include <stdio.h>

int main() {
    printf("GCD of %d and %d equals %d", 21, 77, gcd(21, 77));
    return 0;
}

Overwriting calculate_gcd.c


In [6]:
!gcc -c gcd.c -o gcd.obj
!gcc calculate_gcd.c gcd.obj -o gcd.exe
!./gcd.exe

GCD of 21 and 77 equals 7

- __Note__ that you need exactly one `main` function in one of the files, if you have less or more and compile without special options, `gcc` will fail with error. 

### Options

- `-o` option for giving name to the final generated file.

- `-std` option is for setting standart of `C` language.

- `-Wall` option enables all compiler's warning messages.
  
- `-Werror` option make all warning mesages into error messages.
  
- `Wfatal-errors` option causes the compiler to abort compilation on the first error.

- `-E` option to generate `.c` or `.i` file which will include the content of the source code right after pre-process step.

- `-S` option to generate `.S` assembly file of the source code.
  
- `-c` option to generate `.o` or `.obj` machine-code file withour linking step.
  
- `-g` option for debugging with __gdb__.
  
- `-D` option defines macro to be used during pre-process step.

#### Example

In [15]:
%%file option.c
#include <stdio.h>

int main() {
    printf("TEXT is not defined in programm but TEXT == %s", TEXT);
    return 0;
}

Overwriting option.c


In [16]:
!gcc option.c -DTEXT="\"Hey from shell\"" -o option.exe
!./option.exe

TEXT is not defined in programm but TEXT == Hey from shell

- There are lots of additional options, you can find them [here](https://gcc.gnu.org/onlinedocs/gcc/Option-Summary.html).

### Makefile

- __Makefile__ is used by `make` utility to update, run, compile, delete files and more.

- To use utility `make` you need to install it and write a file, which must strictly have name `Makefile`.

- in `Makefile` we have __targets__ and __dependencies__
  
Here is a template view of __Makefile__

```make
    target1: <prerequisites>
    <TAB> recipe
    target2: ...
```

To run a target we just run in shell `make target_name`.
If you have target called `all` in __Makefile__ it will run in case of using just `make` command

- Note that in both cases Makefile is in the same directory where we run `make` command.

#### Example

```Makefile
say_hello:
    gcc hello_world.c -o hello_world.exe
    ./hello_world.exe
clear:
    rm -f hello_world.exe

```

In [27]:
!make say_hello

gcc hello_world.c -o hello_world.exe
./hello_world.exe
Hello world!


In [28]:
!make clear
!ls | grep "hello_world.exe"

rm -f hello_world.exe > /dev/null
